# PySDMX demo (using sdmxthon internal methods)

In the PySDMX library, even though some methods have been implemented, we are refactoring the internal functions to ensure we can support several SDMX formats.

The notebook displayed here uses the sdmxthon library (whose code has been broadly used in the development of pysdmx parsers and writers for SDMX-ML 2.1, as well as Message class and DataSet class) to showcase the functionalities related to SDMX-ML 2.1.

Some methods may slightly change, but overall the code used here will be similar in PySDMX. A comprehensive example will be uploaded to the documentation when this functionalities are implemented.

Main Links:
- [PyPi]()
- [Documentation]()
- [Github - Source Code and Bug Tracker](https://github.com/bis-med-it/pysdmx)

## Overview of SDMXthon code to be used in PySDMX

SDMXthon is the code base for many functionalities on PySDMX, including the Data Management. It uses Message class as a collection of SDMX objects referring to data or metadata. 

The data is based on Pandas Dataframe. PySDMX may support other libraries like Polars in the future.

The metadata is based on a set of classes to handle the SDMX Information Model, which are already included in PySDMX.

To install SDMXthon in JupyterLite, use this:

In [ ]:
import micropip
await micropip.install("sdmxthon")

## SDMX IM Structures management

PySDMX includes several classes to manage structures, placed in the model package and based on the [SDMX IM](https://sdmx.org/wp-content/uploads/SDMX_2-1-1_SECTION_2_InformationModel_201108.pdf). Here we will use similar classes from the SDMXthon library which already includes the functionality to read and write on SDMX-ML 2.1.

### Item Scheme classes

In [ ]:
from sdmxthon.model.itemScheme import Agency, Codelist, ConceptScheme

cl = Codelist("TEST_CL", version='1.0', maintainer=Agency('TEST'))
cl


In [ ]:
cs = ConceptScheme('TEST_CS', version='1.0', maintainer=Agency('TEST'))
cs

### Data Structure Definition and DataFlow

In [ ]:
from sdmxthon.model.definitions import DataStructureDefinition, DataFlowDefinition

dsd = DataStructureDefinition('TEST_DSD', version='1.0', maintainer=Agency('TEST'))
dsd

In [ ]:
dfd = DataFlowDefinition('TEST_DF', version='1.0', maintainer=Agency('TEST'))
dfd

# The Message class

The message class allows to include all different SDMX objects into a simple class for reading, writing and validation purposes. It is based on the SDMX Message XML object from SDMX-ML format.

In [ ]:
from sdmxthon.utils.parsing_words import CODELISTS, CONCEPTS, DATAFLOWS, DSDS
from sdmxthon import Message, MessageTypeEnum



payload = {
    CODELISTS: {cl.unique_id: cl},
    CONCEPTS: {cs.unique_id: cs},
    DSDS: {dsd.unique_id: dsd}
}

type_ = MessageTypeEnum.Metadata

message = Message(type_, payload)
message.content

# Reading SDMX

PySDMX will support several methods to read from SDMX APIs (api package) or files (io package), enhancing the usability of the library and its functionalities. Still, this functionality is a work in progress, so we will use as demo only the method read_sdmx, which will be included in PySDMX.

In this example, we are reading an SDMX-ML 2.1 Structures file and validating it using the XSD schemas. The output will be the Message class, with all the included SDMX objects parsed.

[Link to example used](https://fmr.meaningfuldata.eu/sdmx/v2/structure/datastructure/BIS/BIS_DER/+/?format=sdmx-2.1&prettyPrint=true)

[Link to FMR DSD]()

In [ ]:
from sdmxthon import read_sdmx

url = 'https://fmr.meaningfuldata.eu/sdmx/v2/structure/datastructure/BIS/BIS_DER/+/?format=sdmx-2.1&prettyPrint=true'

message: Message = read_sdmx(url, validate=True)
message.content['DataStructures']

# Writing Structures to SDMX-ML

PySDMX supports several functionalities to write to SDMX-JSON and SDMX-ML on Structures. This writers are placed in the io package.

In [ ]:
# Method only implemented on SDMXthon, please check the docs to look for the current implementation on PySDMX
message.to_xml(prettyprint=False)

# Uploading Structures to FMR

PySDMX will support the structures upload to Fusion Metadata Registry.

This functionality is still pending to be designed and implemented on PySDMX, as we would like to include it as part of the API package.

If you would like that your platform is also supported, please let us know on [GitHub](https://github.com/bis-med-it/pysdmx)!

[Link to FMR DSD]()

In [ ]:
user = 'XXXXXXXXXXXX'
password = 'XXXXXXXXXXXXXX'

# Method only implemented on SDMXthon, please check the docs to look for the current implementation on PySDMX
message.upload_to_fmr('fmr.meaningfuldata.eu', 443, user=user, password=password, use_https=True)

# Data management

PySDMX allows to read and write data in SDMX formats. All data is managed afterward using the Pandas library. 

In this example, we will load some data on the SDMX-CSV format and write it into SDMX-ML, while validating it using Fusion Metadata Registry.

PySDMX will support the following formats for data:

- Any [Pandas compatible format]()
- SDMX-CSV 1.0
- SDMX-CSV 2.0
- SDMX-JSON 1.0
- SDMX-ML 2.1

## The DataSet class

The DataSet class holds information about the attributes at dataset level and the SDMX IM related attributes (action, reportingBegin...)

It also holds the data (currently only in Pandas DataFrame), to ensure we can manage all data functionalities of the library.

[Link to input data used](https://raw.githubusercontent.com/Meaningful-Data/sdmxthon/master/notebooks/input_data.csv)

In [ ]:
from sdmxthon import Dataset

data_instance = Dataset(unique_id='BIS:BIS_DER(1.0)', structure_type='datastructure')

data_url = 'https://raw.githubusercontent.com/Meaningful-Data/sdmxthon/master/notebooks/input_data.csv'
# Load the data from a CSV file:
data_instance.read_csv(data_url)

metadata = read_sdmx('https://fmr.meaningfuldata.eu/sdmx/v2/structure/datastructure/BIS/BIS_DER/+/?format=sdmx-2.1&prettyPrint=true')
data_instance.structure = metadata.content['DataStructures']['BIS:BIS_DER(1.0)']

## Semantic validation using FMR

PySDMX will support Fusion Metadata Registry´s Semantic Validation compatibility.

In [ ]:
data_instance.fmr_validation('fmr.meaningfuldata.eu', 443, use_https=True)

## Data Writing to SDMX-ML

Writing to SDMX-ML can be done in Structure Specific format or Generic format (deprecated in 3.0)

In [ ]:
data_instance.to_xml(message_type=MessageTypeEnum.StructureSpecificDataSet)

To write using Time Series, we need to set the dimension at observation

In [ ]:
data_instance.set_dimension_at_observation('TIME_PERIOD')
data_instance.to_xml(message_type=MessageTypeEnum.StructureSpecificDataSet)

# PySDMX/SDMXthon in action

[SDMX Insight](https://sdmxinsight.meaningfuldata.eu)

[SDMX Hub (SGDS viewer)](https://sdmxhub.meaningfuldata.eu)